In [ ]:
#Import All Packages Required for Forecasting and Parameter Tuning
import pandas as pd
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
import itertools
import numpy as np
import json


In [ ]:
#Reading in Product Data
products = pd.read_csv('product_data.csv')
products['Week'] = pd.to_datetime(products['Week']).dt.date

In [ ]:
#Initializing Parameter Grid with most likely parameters (based on fbprophet documentation)
#https://facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning
parameters_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'seasonality_mode': ['additive','multiplicative']
}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

In [ ]:
#Function to find best parameters for each error measure
#https://facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning
def getParams(data, productID, all_params, parameters):
    #Prepping Data for Time Series
    productDF = data[['Week',productID]]
    timeSeries = productDF.rename(columns={'Week':'ds',productID:'y'})
    
    #Cutoff dates
    cutoffs = pd.to_datetime(['2020-06-15','2020-12-21','2021-06-14'])
    #Initializing lists
    mae = []
    mse = []
    mape = []
    
    #Looping through parameter grid, trying all combinations
    for params in all_params:
        m = Prophet(**params, weekly_seasonality=True).fit(timeSeries)  # Fit model with given params
        cv = cross_validation(m, cutoffs=cutoffs, horizon='35 days', parallel="processes")
        pm = performance_metrics(cv, rolling_window=1)
        #each loop appends error measure values to list
        mae.append(pm['mae'].values[0])
        mse.append(pm['mse'].values[0])
        mape.append(pm['mape'].values[0])
    
    #getting best combination of parameters for each error measure
    best_mae_params = all_params[np.argmin(mae)]
    best_mse_params = all_params[np.argmin(mse)]
    best_mape_params = all_params[np.argmin(mape)]
    
    #Adding best parameter for each error measure in productID dictionary
    parameters[productID] = {}
    parameters[productID]['mae'] = best_mae_params
    parameters[productID]['mse'] = best_mse_params
    parameters[productID]['mape'] = best_mape_params

In [ ]:
#initializing parameter json structure
parameters = { }
#calls getParams product for each productID
for i in range(1,16):
    productID = 'Product' + str(i)
    getParams(products,productID, all_params, parameters)
#dumping all optimal parameters in json file, for use in gui
with open('parameters_test.json', 'w', encoding = 'utf-8') as jfile:
    json.dump(parameters, jfile,ensure_ascii=False, indent=4)